## Task 1: Optimization model

In [1]:
import gurobipy as gp
from gurobipy import GRB

# Create optimization model for bidding strategy in DA and balancing market
# Probably good to write it as a class or function for easier repeated use

## Task 2: Data collection

Want to determine some features that we believe will be useful predictors of future wind power. Could be previous wind power, features of the weather, season or similar. 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Import relevant data 


# Check correlation between features



# Scale data 
scaler = StandardScaler()
x = scaler.fit_transform(np.random.rand(100, 1))


# Split data into training and test set
split_fraction = 0.8
split_index = int(len(x) * split_fraction)
x_train, x_test = x[:split_index], x[split_index:]

## Task 3: Linear Regression

In [3]:
from sklearn.linear_model import LinearRegression




# Implement gradient descent
theta_gd = ...
# Implement closed form solution
# theta_cfs = np.linalg.inv(x_train.T @ x_train) @ x_train.T @ y_train

# Compare the two solutions
# print(theta_gd)
#print(theta_cfs)


# Evaluate model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

## Task 4: Non-linear regression

In [5]:
from sklearn.preprocessing import PolynomialFeatures

## Task 5: Regularization

In [6]:
from sklearn.linear_model import Ridge, Lasso


## Task 6: Revenue calculation for evaluation

## Task 7 (optional): Suggestion for improvement

In [7]:
from sklearn.cluster import KMeans

## Task 8: Model 2